#CASE FUNCTION

In [0]:
from pyspark.sql.functions import when, expr

In [0]:
schema_characters = "id INTEGER, name STRING, guild STRING, class STRING, level INTEGER, experience INTEGER, health DOUBLE, is_alive BOOLEAN, last_active TIMESTAMP, mentor_id INTEGER"

df_characters = spark.read.format("csv").option("header",True).option("sep",",").schema(schema_characters).load('dbfs:/FileStore/tables/fantasy/characters.csv')

df_characters.show()

+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+
| id|     name|     guild|  class|level|experience|health|is_alive|        last_active|mentor_id|
+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+
|  1|  Aragorn|   Rangers|Warrior|   25|      9000| 120.5|    true|2023-09-20 08:00:00|     null|
|  2|  Legolas|  Mirkwood| Archer|   22|      7500|100.75|    true|2023-09-21 09:00:00|     null|
|  3|    Gimli|    Erebor|Warrior|   20|      6000|130.25|    true|2023-09-22 07:00:00|     null|
|  4|    Frodo| Shirefolk| Hobbit|   12|      2500|  50.5|   false|2023-09-23 10:00:00|        1|
|  5|      Sam| Shirefolk| Hobbit|   13|      2750|  52.5|    true|2023-09-23 10:05:00|        1|
|  6|  Gandalf|    Istari|   Mage|   30|     10000|  90.0|    true|2023-09-20 11:00:00|     null|
|  7|  Boromir|    Gondor|Warrior|   18|      5500| 115.5|   false|2023-09-21 12:00:00|     null|
|  8|    Merry| Shir

##Spark

In [0]:
df_characters.withColumn("tier",
                          when((df_characters.level < 10) | (df_characters["class"] == "Warrior"), "Noob")
                          .when((df_characters.level < 30) & (df_characters.experience > 5000), "Expert")
                          .when(df_characters.level < 30, "Mid")
                          .otherwise("God")
).show()

+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+------+
| id|     name|     guild|  class|level|experience|health|is_alive|        last_active|mentor_id|  tier|
+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+------+
|  1|  Aragorn|   Rangers|Warrior|   25|      9000| 120.5|    true|2023-09-20 08:00:00|     null|  Noob|
|  2|  Legolas|  Mirkwood| Archer|   22|      7500|100.75|    true|2023-09-21 09:00:00|     null|Expert|
|  3|    Gimli|    Erebor|Warrior|   20|      6000|130.25|    true|2023-09-22 07:00:00|     null|  Noob|
|  4|    Frodo| Shirefolk| Hobbit|   12|      2500|  50.5|   false|2023-09-23 10:00:00|        1|   Mid|
|  5|      Sam| Shirefolk| Hobbit|   13|      2750|  52.5|    true|2023-09-23 10:05:00|        1|   Mid|
|  6|  Gandalf|    Istari|   Mage|   30|     10000|  90.0|    true|2023-09-20 11:00:00|     null|   God|
|  7|  Boromir|    Gondor|Warrior|   18|      5500| 115

##SQL

In [0]:
df_characters.withColumn("tier", expr(
    "CASE " +
        "WHEN level < 10 OR class = 'Warrior' THEN 'Noob' " + 
        "WHEN level < 30 AND Experience > 5000 THEN 'Expert' " +
        "WHEN level < 30 THEN 'Mid' " +
        "ELSE 'God' "
    "END"
)).show()

+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+------+
| id|     name|     guild|  class|level|experience|health|is_alive|        last_active|mentor_id|  tier|
+---+---------+----------+-------+-----+----------+------+--------+-------------------+---------+------+
|  1|  Aragorn|   Rangers|Warrior|   25|      9000| 120.5|    true|2023-09-20 08:00:00|     null|  Noob|
|  2|  Legolas|  Mirkwood| Archer|   22|      7500|100.75|    true|2023-09-21 09:00:00|     null|Expert|
|  3|    Gimli|    Erebor|Warrior|   20|      6000|130.25|    true|2023-09-22 07:00:00|     null|  Noob|
|  4|    Frodo| Shirefolk| Hobbit|   12|      2500|  50.5|   false|2023-09-23 10:00:00|        1|   Mid|
|  5|      Sam| Shirefolk| Hobbit|   13|      2750|  52.5|    true|2023-09-23 10:05:00|        1|   Mid|
|  6|  Gandalf|    Istari|   Mage|   30|     10000|  90.0|    true|2023-09-20 11:00:00|     null|   God|
|  7|  Boromir|    Gondor|Warrior|   18|      5500| 115